In [1]:
from pyspark.sql import SparkSession
import os
import time
import datetime
import pyspark.sql.functions as sf
from uuid import *
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import when
from pyspark.sql.functions import col
from pyspark.sql.types import *
from pyspark.sql.functions import lit
from pyspark import SparkConf, SparkContext
from uuid import * 
from uuid import UUID
import time_uuid 
from pyspark.sql import Row
from pyspark.sql.functions import udf
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.window import Window as W

In [6]:
spark = SparkSession.builder.config('spark.jars.packages','com.datastax.spark:spark-cassandra-connector_2.12:3.1.0').getOrCreate()

In [7]:
spark_df = spark.read.format("org.apache.spark.sql.cassandra").options(table='tracking',keyspace='mykeyspace').load()

In [9]:
spark.stop()

In [8]:
spark_df.show()

+--------------------+----+----------+-----------+----+------------+-----+--------------------+---------------+--------------------+----+--------+----+------+----+------------+--------------------+---------+--------------------+------+--------------------+-------------------+------------+-----------+----------+----------+--------+----+--------+
|         create_time| bid|        bn|campaign_id|  cd|custom_track|   de|                  dl|             dt|                  ed|  ev|group_id|  id|job_id|  md|publisher_id|                  rl|       sr|                  ts|    tz|                  ua|                uid|utm_campaign|utm_content|utm_medium|utm_source|utm_term|   v|      vp|
+--------------------+----+----------+-----------+----+------------+-----+--------------------+---------------+--------------------+----+--------+----+------+----+------------+--------------------+---------+--------------------+------+--------------------+-------------------+------------+-----------+-----

In [45]:
spark_df.select('create_time','uid','campaign_id','publisher_id','job_id','custom_track','ts').show()

+--------------------+-------------------+-----------+------------+------+------------+--------------------+
|         create_time|                uid|campaign_id|publisher_id|job_id|custom_track|                  ts|
+--------------------+-------------------+-----------+------------+------+------------+--------------------+
|1a780860-07e6-11e...|1-e1csogrf-l5t49fgf|       NULL|        NULL|  NULL|        NULL|2022-07-20 04:40:...|
|a7b6a3c0-0c8d-11e...|1-t8bqaiy4-l61kszmf|       48.0|         1.0| 273.0|        NULL|2022-07-26 02:50:...|
|c93372f0-08ad-11e...|1-9tivgsfi-l4rae1qy|       NULL|        NULL|  NULL|        NULL|2022-07-21 04:30:...|
|dd5ca050-0d60-11e...|1-347wwfkr-l632u1zk|       93.0|         1.0| 258.0|       click|2022-07-27 04:02:...|
|99bcad50-0cb0-11e...|1-llt0w5dj-l61tvhv3|      222.0|         1.0|1530.0|       click|2022-07-26 07:00:...|
|a5662fd0-0073-11e...|1-rrc3k5vd-l4o0b4yy|       NULL|        NULL|  NULL|       click|2022-07-10 17:13:...|
|3b681680-0c8e-11e.

In [46]:
df = spark_df.select('create_time','ts','job_id','custom_track','bid','campaign_id','group_id','publisher_id')

In [47]:
df.show()

+--------------------+--------------------+------+------------+----+-----------+--------+------------+
|         create_time|                  ts|job_id|custom_track| bid|campaign_id|group_id|publisher_id|
+--------------------+--------------------+------+------------+----+-----------+--------+------------+
|44e04070-02ca-11e...|2022-07-13 16:38:...|  NULL|       click|NULL|       NULL|    NULL|        NULL|
|62c25a80-0d81-11e...|2022-07-27 07:54:...|  NULL|        NULL|NULL|       NULL|    NULL|        NULL|
|484882a0-0c97-11e...|2022-07-26 03:59:...|  NULL|        NULL|NULL|       NULL|    NULL|        NULL|
|ad1fbc90-0844-11e...|2022-07-20 15:57:...|  NULL|        NULL|NULL|       NULL|    NULL|        NULL|
|37ccc9cc-9455-11e...| 2023-01-14 21:48:44|1592.0|       click| 1.0|       10.0|    32.0|        28.0|
|794157a0-0563-11e...|2022-07-17 00:00:...|  NULL|        NULL|NULL|       NULL|    NULL|        NULL|
|62d750a0-0ae5-11e...|2022-07-24 00:13:...|  NULL|        NULL|NULL|     

In [48]:
df = df.filter((df.job_id.isNotNull())&(df.custom_track.isNotNull()))

In [49]:
df.show()

+--------------------+--------------------+------+------------+---+-----------+--------+------------+
|         create_time|                  ts|job_id|custom_track|bid|campaign_id|group_id|publisher_id|
+--------------------+--------------------+------+------------+---+-----------+--------+------------+
|f1d89630-0c00-11e...|2022-07-25 10:02:...|1527.0|       click|0.0|      222.0|    NULL|         1.0|
|01461160-0cb0-11e...|2022-07-26 06:56:...| 273.0|       click|0.0|       48.0|    NULL|         1.0|
|f3933860-0b59-11e...|2022-07-24 14:07:...| 188.0|       click|1.0|       48.0|    34.0|         1.0|
|123749e0-0c00-11e...|2022-07-25 09:56:...|1530.0|       click|0.0|      222.0|    NULL|         1.0|
|fa94ab90-0c8e-11e...|2022-07-26 02:59:...|1531.0|       click|0.0|      222.0|    NULL|         1.0|
|b30647e0-0c00-11e...|2022-07-25 10:01:...| 273.0|       click|0.0|       48.0|    NULL|         1.0|
|50745280-0bff-11e...|2022-07-25 09:51:...|1534.0|       click|0.0|      222.0|   

# TS = time in user machine
# Create time = time in data lake

In [50]:
spark_time = df.select("create_time").collect()

In [51]:
spark_time

[Row(create_time='46a82420-0b5a-11ed-95ee-a4a8753bccff'),
 Row(create_time='380246c0-0c00-11ed-a478-80f74b7531f9'),
 Row(create_time='36a46960-0c8d-11ed-99fc-bc1d1e5f5827'),
 Row(create_time='29c83770-0c8e-11ed-a22f-b1e0d0cbb696'),
 Row(create_time='46c95740-fea2-11ec-8d3f-432d1ce501df'),
 Row(create_time='d94e1860-fe77-11ec-aff4-9c748625f181'),
 Row(create_time='7664a430-0b5b-11ed-ba04-dd2a3507fcec'),
 Row(create_time='df5f8770-0bfb-11ed-b086-1504ea124044'),
 Row(create_time='471f79f0-0bfd-11ed-a294-807a7a3361fc'),
 Row(create_time='01e326b2-9455-11ed-8bb9-398d6d80d135'),
 Row(create_time='23f35380-0cb0-11ed-9f28-4cb95a960b16'),
 Row(create_time='0aeade00-0b5a-11ed-a541-edb8489291c1'),
 Row(create_time='d29b4f70-0cae-11ed-85b9-25d04b70d557'),
 Row(create_time='3e6463d4-9456-11ed-8d55-f6cdbf83007e'),
 Row(create_time='0b66d760-0cb0-11ed-9b41-49bb146594c7'),
 Row(create_time='1a5d88f8-9456-11ed-a092-97e8410ef2b3'),
 Row(create_time='5d8378e0-0c8e-11ed-8a1c-e7eafaf46eb6'),
 Row(create_ti

In [52]:
normal_time = []
for i in range(len(spark_time)):
    a = time_uuid.TimeUUID(bytes=UUID(df.select('create_time').collect()[i][0]).bytes).get_datetime().strftime(('%Y-%m-%d %H:%M:%S'))
    normal_time.append(a)


In [53]:
normal_time

['2022-07-08 09:29:08',
 '2022-07-25 10:02:54',
 '2022-07-08 09:54:16',
 '2022-07-26 07:05:52',
 '2022-07-26 02:53:40',
 '2022-07-08 09:27:40',
 '2023-01-14 21:49:14',
 '2022-07-26 07:03:13',
 '2022-07-26 03:02:38',
 '2022-07-26 07:04:17',
 '2022-07-26 06:53:00',
 '2022-07-26 02:48:34',
 '2022-07-25 10:01:20',
 '2022-07-27 04:19:21',
 '2022-07-26 06:45:19',
 '2022-07-25 09:52:35',
 '2022-07-26 06:53:19',
 '2022-07-27 04:03:13',
 '2022-07-25 09:56:08',
 '2022-07-25 09:43:04',
 '2022-07-24 14:07:38',
 '2022-07-26 07:02:35',
 '2022-07-25 09:36:21',
 '2022-07-27 04:14:42',
 '2022-07-24 10:06:20',
 '2023-01-14 21:55:35',
 '2022-07-26 02:56:39',
 '2022-07-27 04:20:56',
 '2022-07-08 09:43:15',
 '2022-07-26 02:48:22',
 '2022-07-26 02:59:59',
 '2022-07-08 09:46:31',
 '2022-07-27 04:05:14',
 '2022-07-26 02:54:20',
 '2022-07-24 14:08:09',
 '2022-07-24 14:17:34',
 '2022-07-08 09:30:51',
 '2022-07-25 09:56:08',
 '2023-01-14 21:56:05',
 '2022-07-24 14:17:24',
 '2022-07-27 04:14:09',
 '2022-07-26 02:

In [54]:
spark_timeuuid = []
for i in range(len(spark_time)):
    spark_timeuuid.append(spark_time[i][0])

In [55]:
time_data = spark.createDataFrame(zip(spark_timeuuid,normal_time),['create_time','ts'])

In [56]:
time_data.show()

+--------------------+-------------------+
|         create_time|                 ts|
+--------------------+-------------------+
|46a82420-0b5a-11e...|2022-07-08 09:29:08|
|380246c0-0c00-11e...|2022-07-25 10:02:54|
|36a46960-0c8d-11e...|2022-07-08 09:54:16|
|29c83770-0c8e-11e...|2022-07-26 07:05:52|
|46c95740-fea2-11e...|2022-07-26 02:53:40|
|d94e1860-fe77-11e...|2022-07-08 09:27:40|
|7664a430-0b5b-11e...|2023-01-14 21:49:14|
|df5f8770-0bfb-11e...|2022-07-26 07:03:13|
|471f79f0-0bfd-11e...|2022-07-26 03:02:38|
|01e326b2-9455-11e...|2022-07-26 07:04:17|
|23f35380-0cb0-11e...|2022-07-26 06:53:00|
|0aeade00-0b5a-11e...|2022-07-26 02:48:34|
|d29b4f70-0cae-11e...|2022-07-25 10:01:20|
|3e6463d4-9456-11e...|2022-07-27 04:19:21|
|0b66d760-0cb0-11e...|2022-07-26 06:45:19|
|1a5d88f8-9456-11e...|2022-07-25 09:52:35|
|5d8378e0-0c8e-11e...|2022-07-26 06:53:19|
|680be5d0-fea2-11e...|2022-07-27 04:03:13|
|2d006f40-0d63-11e...|2022-07-25 09:56:08|
|6dc68690-0b5b-11e...|2022-07-25 09:43:04|
+----------

In [57]:
df = df.drop('ts')

In [58]:
result = df.join(time_data,on='create_time',how='inner')

In [59]:
result.show()

+--------------------+------+------------+---+-----------+--------+------------+-------------------+
|         create_time|job_id|custom_track|bid|campaign_id|group_id|publisher_id|                 ts|
+--------------------+------+------------+---+-----------+--------+------------+-------------------+
|b83be620-0c00-11e...|1530.0|       click|0.0|      222.0|    NULL|         1.0|2022-07-25 09:43:04|
|46a82420-0b5a-11e...| 188.0|       click|1.0|       48.0|    34.0|         1.0|2022-07-08 09:29:08|
|3e6463d4-9456-11e...| 782.0|   qualified|0.0|       48.0|    10.0|        17.0|2022-07-27 04:19:21|
|903a2840-0d62-11e...|1529.0|       click|0.0|      222.0|    NULL|         1.0|2022-07-27 04:14:09|
|435afd00-0caf-11e...| 440.0|       click|7.0|        1.0|    NULL|         1.0|2022-07-26 02:56:39|
|cf600a40-0d61-11e...|  98.0|       click|2.0|        4.0|    NULL|         1.0|2022-07-25 09:49:56|
|7f117cc0-0b38-11e...| 188.0|       click|1.0|       48.0|    34.0|         1.0|2022-07-27 

In [60]:
click_data = result.filter(df.custom_track == 'click')
click_data.show()

+--------------------+------+------------+---+-----------+--------+------------+-------------------+
|         create_time|job_id|custom_track|bid|campaign_id|group_id|publisher_id|                 ts|
+--------------------+------+------------+---+-----------+--------+------------+-------------------+
|b83be620-0c00-11e...|1530.0|       click|0.0|      222.0|    NULL|         1.0|2022-07-25 09:43:04|
|46a82420-0b5a-11e...| 188.0|       click|1.0|       48.0|    34.0|         1.0|2022-07-08 09:29:08|
|903a2840-0d62-11e...|1529.0|       click|0.0|      222.0|    NULL|         1.0|2022-07-27 04:14:09|
|435afd00-0caf-11e...| 440.0|       click|7.0|        1.0|    NULL|         1.0|2022-07-26 02:56:39|
|cf600a40-0d61-11e...|  98.0|       click|2.0|        4.0|    NULL|         1.0|2022-07-25 09:49:56|
|7f117cc0-0b38-11e...| 188.0|       click|1.0|       48.0|    34.0|         1.0|2022-07-27 04:11:31|
|1fdf7250-fea1-11e...|  98.0|       click|2.0|        4.0|    NULL|         1.0|2022-07-27 

In [61]:
clicks_data = click_data.fillna(0)

In [62]:
clicks_data.show()

+--------------------+------+------------+---+-----------+--------+------------+-------------------+
|         create_time|job_id|custom_track|bid|campaign_id|group_id|publisher_id|                 ts|
+--------------------+------+------------+---+-----------+--------+------------+-------------------+
|b83be620-0c00-11e...|1530.0|       click|0.0|      222.0|     0.0|         1.0|2022-07-25 09:43:04|
|46a82420-0b5a-11e...| 188.0|       click|1.0|       48.0|    34.0|         1.0|2022-07-08 09:29:08|
|903a2840-0d62-11e...|1529.0|       click|0.0|      222.0|     0.0|         1.0|2022-07-27 04:14:09|
|435afd00-0caf-11e...| 440.0|       click|7.0|        1.0|     0.0|         1.0|2022-07-26 02:56:39|
|cf600a40-0d61-11e...|  98.0|       click|2.0|        4.0|     0.0|         1.0|2022-07-25 09:49:56|
|7f117cc0-0b38-11e...| 188.0|       click|1.0|       48.0|    34.0|         1.0|2022-07-27 04:11:31|
|1fdf7250-fea1-11e...|  98.0|       click|2.0|        4.0|     0.0|         1.0|2022-07-27 

In [63]:
clicks_data.registerTempTable('clicks')

In [64]:
click_output = spark.sql(""" select job_id,date(ts) as Date,hour(ts) as Hour, publisher_id, campaign_id, group_id, avg(bid) as bid_set, count(*) as clicks,sum(bid) as spend_hour from clicks 
          group by job_id,date(ts),hour(ts),publisher_id,campaign_id,group_id""")

In [65]:
conversion_data = result.filter(result.custom_track == 'conversion')

In [66]:
conversion_data = conversion_data.fillna(0)

In [67]:
conversion_data.show()

+--------------------+------+------------+---+-----------+--------+------------+-------------------+
|         create_time|job_id|custom_track|bid|campaign_id|group_id|publisher_id|                 ts|
+--------------------+------+------------+---+-----------+--------+------------+-------------------+
|1a5b4354-9456-11e...|1119.0|  conversion|1.0|      223.0|    41.0|         3.0|2023-01-14 21:55:35|
|3e5abb86-9456-11e...|1686.0|  conversion|0.0|        1.0|    32.0|        32.0|2022-07-27 04:03:49|
|f6564ba2-9455-11e...|  81.0|  conversion|1.0|        1.0|    10.0|        11.0|2022-07-27 04:07:08|
|2c597e04-9456-11e...| 672.0|  conversion|0.0|       61.0|    11.0|        15.0|2022-07-25 10:01:39|
|1a37e634-9456-11e...| 458.0|  conversion|0.0|       48.0|    25.0|        17.0|2022-07-08 09:42:26|
|1a4f72ae-9456-11e...|1723.0|  conversion|0.0|       79.0|    32.0|        13.0|2022-07-27 03:59:52|
|01ea3eca-9455-11e...| 858.0|  conversion|1.0|        1.0|    32.0|        15.0|2022-07-26 

In [68]:
conversion_data.registerTempTable("conversion")

In [69]:
conversion_output = spark.sql("""select job_id,date(ts) as Date,hour(ts) as Hour, publisher_id, campaign_id, group_id, count(*) as conversion from conversion 
          group by job_id,date(ts),hour(ts),publisher_id,campaign_id,group_id""")

In [70]:
conversion_output.show()

+------+----------+----+------------+-----------+--------+----------+
|job_id|      Date|Hour|publisher_id|campaign_id|group_id|conversion|
+------+----------+----+------------+-----------+--------+----------+
|1119.0|2023-01-14|  21|         3.0|      223.0|    41.0|         1|
|1822.0|2022-07-26|   7|        22.0|       55.0|    27.0|         1|
|  81.0|2022-07-27|   4|        11.0|        1.0|    10.0|         1|
| 969.0|2022-07-27|   4|        13.0|        5.0|    35.0|         1|
| 458.0|2022-07-08|   9|        17.0|       48.0|    25.0|         1|
| 672.0|2022-07-25|  10|        15.0|       61.0|    11.0|         1|
| 858.0|2022-07-26|   2|        15.0|        1.0|    32.0|         1|
|1616.0|2022-07-08|   9|        31.0|       48.0|    26.0|         1|
| 355.0|2022-07-08|   9|         2.0|       61.0|    21.0|         1|
|1806.0|2022-07-26|   7|        17.0|      101.0|    10.0|         1|
|2009.0|2022-07-26|   7|        31.0|       48.0|    10.0|         1|
| 125.0|2022-07-08| 

In [71]:
qualified_data = result.filter(result.custom_track == 'qualified')
qualified_data = qualified_data.fillna(0)
qualified_data.registerTempTable("qualified")
qualified_output = spark.sql("""select job_id,date(ts) as Date,hour(ts) as Hour, publisher_id, campaign_id, group_id, count(*) as qualified from qualified 
          group by job_id,date(ts),hour(ts),publisher_id,campaign_id,group_id""")
qualified_output.show()


+------+----------+----+------------+-----------+--------+---------+
|job_id|      Date|Hour|publisher_id|campaign_id|group_id|qualified|
+------+----------+----+------------+-----------+--------+---------+
| 263.0|2022-07-27|   4|         3.0|       59.0|    26.0|        1|
| 671.0|2022-07-08|   9|        34.0|       12.0|    17.0|        1|
| 192.0|2022-07-08|   9|        15.0|       51.0|    41.0|        1|
|1086.0|2022-07-25|   9|        15.0|       48.0|    13.0|        1|
|1644.0|2022-07-26|   2|        22.0|      141.0|    10.0|        1|
| 505.0|2022-07-25|   9|         2.0|       53.0|    11.0|        1|
| 339.0|2022-07-27|   4|        29.0|       12.0|    41.0|        1|
| 496.0|2022-07-24|  14|        38.0|        1.0|    17.0|        1|
| 379.0|2022-07-26|   6|         3.0|       12.0|    22.0|        1|
| 782.0|2022-07-27|   4|        17.0|       48.0|    10.0|        1|
| 245.0|2022-07-26|   7|        24.0|        1.0|    13.0|        1|
|1400.0|2022-07-25|   9|        30

In [72]:
unqualified_data = result.filter(result.custom_track == 'unqualified')
unqualified_data = unqualified_data.fillna(0)
unqualified_data.registerTempTable("unqualified")
unqualified_output = spark.sql("""select job_id,date(ts) as Date,hour(ts) as Hour, publisher_id, campaign_id, group_id, count(*) as unqualified from unqualified 
          group by job_id,date(ts),hour(ts),publisher_id,campaign_id,group_id""")
unqualified_output.show()


+------+----------+----+------------+-----------+--------+-----------+
|job_id|      Date|Hour|publisher_id|campaign_id|group_id|unqualified|
+------+----------+----+------------+-----------+--------+-----------+
| 285.0|2022-07-26|   2|         9.0|       57.0|    41.0|          1|
|2002.0|2022-07-26|   7|        33.0|       15.0|    17.0|          1|
| 311.0|2022-07-26|   6|        12.0|       53.0|    14.0|          1|
|1197.0|2022-07-26|   2|        17.0|       57.0|    41.0|          1|
|1791.0|2022-07-27|   4|        10.0|        5.0|    10.0|          1|
| 512.0|2023-01-14|  21|        17.0|       61.0|    35.0|          1|
| 463.0|2022-07-24|  14|         3.0|        1.0|    17.0|          1|
|1130.0|2022-07-25|   9|        30.0|      117.0|    19.0|          1|
|1414.0|2022-07-26|   6|        24.0|       10.0|    10.0|          1|
| 760.0|2022-07-26|   3|        37.0|       13.0|    32.0|          1|
|1688.0|2022-07-26|   6|        17.0|       61.0|    26.0|          1|
| 885.

In [73]:
final_output = click_output.join(conversion_output,['job_id','date','hour','publisher_id','campaign_id','group_id'],'full').\
join(qualified_output,['job_id','date','hour','publisher_id','campaign_id','group_id'],'full').\
join(unqualified_output,['job_id','date','hour','publisher_id','campaign_id','group_id'],'full')

In [74]:
company_df = spark.read.csv('D:\\LEARNING\\Study_DE\\Gen5\\Gen5\\Class 4 - Data Pipeline Project\\MySQL\\job.csv',header = True)

In [75]:
company_df.show()

+---+----------+--------------------+----------------+--------------------+---------+----------------+-----------+-------------+-----------+--------------+-------------+-------------+-------------+------+------+-----+--------------------+----------------------+-------------------------+------------+----------+--------+----------+--------+--------+-----------+----------+--------------+------+
| id|created_by|        created_date|last_modified_by|  last_modified_date|is_active|           title|description|work_schedule|radius_unit|location_state|location_list|role_location|resume_option|budget|status|error|template_question_id|template_question_name|question_form_description|redirect_url|start_date|end_date|close_date|group_id|minor_id|campaign_id|company_id|history_status|ref_id|
+---+----------+--------------------+----------------+--------------------+---------+----------------+-----------+-------------+-----------+--------------+-------------+-------------+-------------+------+------

In [76]:
company_df = company_df.select('id','company_id','campaign_id','group_id')

In [77]:
company_df = company_df.withColumnRenamed('id','job_id')

In [78]:
company_df = company_df.drop('campaign_id','group_id')

In [79]:
final_output = final_output.join(company_df,on=['job_id'],how='left')

In [80]:
final_output.show()

+------+----------+----+------------+-----------+--------+-------+------+----------+----------+---------+-----------+----------+
|job_id|      Date|Hour|publisher_id|campaign_id|group_id|bid_set|clicks|spend_hour|conversion|qualified|unqualified|company_id|
+------+----------+----+------------+-----------+--------+-------+------+----------+----------+---------+-----------+----------+
|  76.0|2022-07-27|   4|        33.0|       54.0|    10.0|   NULL|  NULL|      NULL|      NULL|     NULL|          1|         1|
|  81.0|2022-07-27|   4|        11.0|        1.0|    10.0|   NULL|  NULL|      NULL|         1|     NULL|       NULL|         1|
|  90.0|2022-07-26|   2|        33.0|      198.0|    10.0|   NULL|  NULL|      NULL|      NULL|     NULL|          1|         1|
|  98.0|2022-07-08|   9|         1.0|        4.0|     0.0|    2.0|    19|      38.0|      NULL|     NULL|       NULL|         1|
|  98.0|2022-07-24|  10|         1.0|        4.0|     0.0|    2.0|     7|      14.0|      NULL|  

In [81]:
final_output = final_output.select('job_id','date','hour','publisher_id','company_id','campaign_id','group_id','unqualified','qualified','conversion','clicks','bid_set','spend_hour')

In [82]:
final_output = final_output.withColumnRenamed('date','dates').withColumnRenamed('hour','hours').withColumnRenamed('qualified','qualified_application').\
    withColumnRenamed('unqualified','disqualified_application')

In [83]:
final_output = final_output.withColumn('sources',lit('Cassandra'))

In [84]:
final_output.columns

['job_id',
 'dates',
 'hours',
 'publisher_id',
 'company_id',
 'campaign_id',
 'group_id',
 'disqualified_application',
 'qualified_application',
 'conversion',
 'clicks',
 'bid_set',
 'spend_hour',
 'sources']

In [87]:
final_output.show()

+------+----------+-----+------------+----------+-----------+--------+------------------------+---------------------+----------+------+-------+----------+---------+
|job_id|     dates|hours|publisher_id|company_id|campaign_id|group_id|disqualified_application|qualified_application|conversion|clicks|bid_set|spend_hour|  sources|
+------+----------+-----+------------+----------+-----------+--------+------------------------+---------------------+----------+------+-------+----------+---------+
|  76.0|2022-07-27|    4|        33.0|         1|       54.0|    10.0|                       1|                 NULL|      NULL|  NULL|   NULL|      NULL|Cassandra|
|  81.0|2022-07-27|    4|        11.0|         1|        1.0|    10.0|                    NULL|                 NULL|         1|  NULL|   NULL|      NULL|Cassandra|
|  90.0|2022-07-26|    2|        33.0|         1|      198.0|    10.0|                       1|                 NULL|      NULL|  NULL|   NULL|      NULL|Cassandra|
|  98.0|20

In [85]:
def import_To_MySql(data,dbname,table_name,username,password):
    
    data.write \
    .mode("append") \
    .format("jdbc") \
    .option("driver", "com.mysql.jdbc.Driver") \
    .option("url", "jdbc:mysql://localhost:3306/"+ dbname) \
    .option("dbtable", table_name) \
    .option("user", username) \
    .option("batchsize", 2000) \
    .option("password", password) \
    .save()
    
    print("Import to MySQL completed")

In [86]:
import_To_MySql(final_output,"storagedb","event_table","root",1234)

Import to MySQL completed
